In [1]:
cd ../

/Users/creator/Projects/CS6241


In [2]:
from fairrecs.preprocessor import preprocess_yow
from fairrecs.solver import Solver
from fairrecs.dp_solver import DPSolver
import numpy as np

In [3]:
dataset = preprocess_yow('datasets/yow_userstudy_raw.csv')
dataset.head()

,EventOnScroll,MSecForDownArrow,TimeOnVScroll,relevant,NumOfPageUp,TimeOnMouse,NumOfPageDown,ClickOnWindow,log_id,serverTimeVisit,...,readability,TimeOnHScroll,novelty,user_like,DOC_ID,TimeOnPage,TimeVisit,user_id,NumOfDownArrow,authority
9780,0.0,0.0,0.0,0.521603,0,2156,0.0,0.0,5851,2004-05-02 18:40,...,1.0,0.0,2.0,3,295842,106781.0,2004-05-02 18:40,92,0.0,1.0
9781,0.0,0.0,0.0,0.407763,0,3934,0.0,0.0,5855,2004-05-02 18:42,...,1.0,0.0,4.0,4,282679,129141.0,2004-05-02 18:42,92,0.0,1.0
9782,0.0,0.0,0.0,0.398252,0,4280,0.0,1.0,5861,2004-05-02 18:45,...,1.0,0.0,2.0,3,295838,107500.0,2004-05-02 18:45,92,0.0,1.0
9784,0.0,0.0,0.0,0.396782,0,3339,0.0,0.0,5868,2004-05-02 18:50,...,1.0,0.0,2.0,4,281594,147469.0,2004-05-02 18:50,92,0.0,1.0
9785,0.0,0.0,0.0,0.623912,0,4561,0.0,1.0,5872,2004-05-02 18:53,...,1.0,0.0,2.0,4,294299,178390.0,2004-05-02 18:53,92,0.0,1.0


In [4]:
solver = Solver(dataset['relevant'].to_numpy())
P = solver.solve()
print(P)

[[ 2.71504382e-05  2.73718679e-05  2.69672862e-05 ...  2.11954415e-05
   1.02845459e-05  1.97640279e-05]
 [-2.32707032e-06 -2.05812175e-06 -2.44584578e-06 ... -6.53935548e-06
  -1.89897361e-05 -9.60926161e-06]
 [-1.96474022e-05 -1.93744836e-05 -1.97607992e-05 ... -6.77693267e-05
  -3.63998654e-05 -2.69208864e-05]
 ...
 [-4.19368047e-06 -4.06864789e-06 -4.50742693e-06 ... -1.03593453e-05
  -2.12706242e-05 -1.17915098e-05]
 [-9.82319919e-06 -9.53213632e-06 -9.91201518e-06 ...  2.25215439e-01
   2.48874452e-01  2.52732036e-01]
 [-3.40664918e-06 -3.30170544e-06 -3.74761112e-06 ... -9.61621655e-06
  -2.05275754e-05 -1.10485375e-05]]


In [5]:
solver = DPSolver(dataset['relevant'].to_numpy(), dataset['RSS_ID'].to_numpy())
P = solver.solve()
print(P)

KeyError: 170

In [ ]:
dataset['RSS_ID'].to_numpy().reshape(-1, 1)